In [1]:
import time
import torch
import sbibm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import io

In [2]:
task_name = "two_moons"
device = "cpu"
task = sbibm.get_task(task_name)
prior = task.get_prior_dist()
simulator = task.get_simulator()

In [3]:
cached_fn = os.path.join("trained", f"{task_name}.nf")
with open(cached_fn, "rb") as f:
    encoder = pickle.load(f)
encoder.to(device)

Flow(
  (_transform): CompositeTransform(
    (_transforms): ModuleList(
      (0): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=3, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_layer): Linear(in_features=2, out_features=50, bias=True)
              (linear_layers): ModuleList(
                (0-1): 2 x Linear(in_features=50, out_features=50, bias=True)
              )
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (final_layer): Linear(in_features=50, out_features=29, bias=True)
        )
      )
      (1): LULinear()
      (2): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=3, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_layer):

In [25]:
test_sim = 1
test_theta = prior.sample((test_sim,))
test_x = simulator(test_theta)
A = np.hstack([test_theta, test_theta]).reshape((2, 2))
B = np.hstack([test_theta]).reshape((2, 1))

In [26]:
t = np.linspace(0, 10, 100)
K = np.random.rand(1, 2) # random gain matrix

In [27]:
X = np.zeros((len(t), 2))
U = np.zeros((len(t), 1))
X[0] = np.random.random()

for t_idx in range(len(t)):
    U[t_idx] = K @ X[t_idx]
    if t_idx != len(t) - 1:
        X[t_idx + 1] = (A - B @ K) @ X[t_idx]

# Combine state and control input into a matrix
data_matrix = np.hstack([X, U]).T

In [25]:
# done: 1. generate C
# done: 2. learn generative model for C^ (seems like it'll be identical since we can just consider C to be a reshaped vector)
# done: 3. simulate trajectories w/ C/C_hat (sort of separate, but can do concurrently)
# 4. recover C^ from trajetories 
# 5. learn optimal controller for C_hat

(1, 18)